## 🟢 1. Execução Interativa em Terminal

**🔹 Enunciado:**  
Implemente um chatbot simples em terminal que responde perguntas em português, utilizando exemplos explícitos no prompt.

**🧩 Implementação:**  
O código carrega o modelo `TinyLlama-1.1B-Chat` da Hugging Face, define um prompt com instruções e exemplos, recebe uma pergunta via `input()` e gera uma resposta com `transformers`.

**🚀 Resultados Esperados:**
- Executado em ambiente local com suporte CUDA ou CPU.
- Geração de respostas coerentes em português com base em poucos-shots no prompt.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

In [ ]:
pergunta = input("Pergunta: ")

In [ ]:
# Prompt explícito com exemplos
prompt = f"""<|system|>Você é um assistente útil que responde sempre em português de forma clara e objetiva.<|end|>
<|user|>Qual é a capital da França?<|end|>
<|assistant|>A capital da França é Paris.<|end|>
<|user|>Quem descobriu o Brasil?<|end|>
<|assistant|>O Brasil foi descoberto por Pedro Álvares Cabral em 1500.<|end|>
<|user|>{pergunta}<|end|>
<|assistant|>"""

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=150,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

In [ ]:
saida = tokenizer.decode(outputs[0], skip_special_tokens=True)
resposta = saida.split("<|assistant|>")[-1].strip()
print("Resposta:", resposta)